# Loading Data into TinyFlux

This notebook demonstrates how to build a TinyFlux database from a CSV and a JSON datasource.

AQI data comes from the [EPA website](https://aqs.epa.gov/aqsweb/airdata/download_files.html
) and CBSA location data comes from the [US Census Bureau](https://catalog.data.gov/dataset/tiger-line-shapefile-2019-nation-u-s-current-metropolitan-statistical-area-micropolitan-statist).

### Import TinyFlux

In [1]:
import csv
from datetime import datetime
import json
from zoneinfo import ZoneInfo

from tinyflux import TinyFlux, Point

In [2]:
# Files.
INPUT_FILE = "daily_aqi_by_cbsa_ca_2019-2020.csv"
DB_FILE = "ca_aqi_2019-2020.tinyflux"

### Read CSV into Memory

In [3]:
col_names = []
data = []

with open(INPUT_FILE) as f:
    r = csv.reader(f)
    for i, row in enumerate(r):
        if i == 0:
            col_names = row
        else:
            data.append(row)

print(f"Csv loaded, {len(data)} records.\n")
print(f'Column names: {", ".join(col_names)}\n')

Csv loaded, 24608 records.

Column names: CBSA, CBSA Code, Date, AQI, Category, Defining Parameter, Defining Site, Number of Sites Reporting



### Initialize TinyFlux DB.

In [4]:
db = TinyFlux(DB_FILE)

### Initialize TinyFlux Points

In [5]:
points = []

for row in data:
    # Measurement name, a string.
    measurement = "aqi"
    
    # Datetime object that is "timezone-aware".
    ts_naive = datetime.strptime(row[2], "%Y-%m-%d")
    ts_aware = ts_naive.replace(tzinfo=ZoneInfo("US/Pacific"))

    # Tags as a dict of string/string key values.
    tags = {
        "cbsa": str(row[0]),
        "cbsa_code": str(row[1]),
        "category": str(row[4]),
        "defining_parameter": str(row[5]),
        "defining_site": str(row[6]),
    }
    
    # Fields as a dict of string/numeric key values.
    fields = {
        "aqi": int(row[3]),
        "number_of_sites_reporting": int(row[7]),
    }

    # Initialize the Point with the above attributes.
    p = Point(
        measurement=measurement,
        time=ts_aware,
        tags=tags,
        fields=fields,
    )
    
    points.append(p)

### Insert Points into TinyFlux

In [6]:
db.insert_multiple(points)

24608

In [7]:
print(f"TinyFlux database {DB_FILE} has {len(db)} records.")

TinyFlux database ca_aqi_2019-2020.tinyflux has 24608 records.


## Read Geospatial Data

In [ ]:
data = json.load(open("cbsa_ca_2019.geojson"))

### Insert geospatial data

In [8]:
for feature in data['features']:
    # Measurement name, a string.
    measurement = "locations"

    # Datetime object that is "timezone-aware".
    ts = datetime(2019, 1, 1, tzinfo=ZoneInfo("US/Pacific"))

    # Tags as a dict of string/string key values.
    tags = {
        "cbsa": feature['properties']['NAME'],
        "cbsa_code": feature['properties']['CBSAFP'],
    }
    
    # Fields as a dict of string/numeric key values.
    fields = {
        "latitude": feature['geometry']['coordinates'][1],
        "longitude": feature['geometry']['coordinates'][0],
    }

    # Initialize the Point with the above attributes.
    p = Point(
        measurement=measurement,
        time=ts_aware,
        tags=tags,
        fields=fields,
    )
        
    db.insert(p)

In [9]:
print(f"TinyFlux database {DB_FILE} has {len(db)} records.")

TinyFlux database ca_aqi_2019-2020.tinyflux has 24642 records.
